In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def load_dataset(batch_size):
    transformer = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))]) #ToTensor transform the image to (C,H,W) (0,1) form and Normalize limits the tensor value between (-1,1)
    train_dataset = torchvision.datasets.MNIST(root='./data',train=True,transform=transformer,download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data',train=False,transform=transformer,download=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) #improve the generalization
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader



In [6]:
class myCNN(nn.Module):
    def __init__(self):
        super(myCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64*7*7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
def train(model,criterion,optimizer,num_epoch,dataset):
    for epoch in range(num_epoch):
        model.train()
        for images, labels in dataset:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

def test(model, criterion, optimizer, dataset):
    model.eval()
    total_num = 0
    acc_num = 0
    for images, labels in dataset:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_num += labels.size(0)
        acc_num += (labels==predicted).sum().item()
        
    print(f"Accuracy on the test dataset:{100*acc_num/total_num}%")

In [9]:
import torch.optim as optim

model = myCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

train_dataset, test_dataset = load_dataset(64)

train(model,criterion,optimizer,5,train_dataset)
test(model,criterion,optimizer,test_dataset)

Accuracy on the test dataset:97.18%
